In [5]:
# short  << 2k
short_prompt = "Here is an introduction of the city LA:"

# long 
middle_prompt ="""
Text
---
Large language models (LLMs) typically come with a pre-defined context window size. For example, inputs to LLaMA models (Touvron et al., 2023) must be fewer than 2048 tokens. This pre-set
context window limit is frequently exceeded in applications such as conducting long conversations,
summarizing long documents, or executing long-term planning. For these applications, LLMs with
longer context windows are preferred. However, training an LLM from scratch with long context
windows requires significant investments. This naturally leads to a question: Can we extend the
context window of an existing pre-trained LLM?
One straightforward approach is to fine-tune an existing pre-trained Transformer with a longer context window. However, empirically, we found that models trained this way adapt to long context
windows very slowly. After training for more than 10000 batches, the effective context window
saw a minimal increase, moving from 2048 to 2560 (Table 4). This suggests that such method is
inefficient for extending to substantially longer context windows.
While certain techniques such as ALiBi (Press et al., 2022) and LeX (Sun et al., 2022) enable length
extrapolation of Transformers, i.e. train on short context windows and inference on longer ones,
many existing pre-trained LLMs, including LLaMA (Touvron et al., 2023), use positional encodings
that have weak extrapolation properties (e.g., RoPE (Su et al., 2021)). Therefore, the applicability
of these techniques for extending the context window sizes of such LLMs remains limited.
In this work, we introduce Position Interpolation to enable context window extensions for certain
existing pre-trained LLMs, including LLaMA. The key idea is, instead of extrapolation, we directly
down-scale the position indices so that the maximum position index matches the previous context
window limit in the pre-training stage. See Figure 1 for an illustration. In other words, to accommodate more input tokens, we interpolate the position encodings at neighboring integer positions,
utilizing the fact that position encodings can be applied on non-integer positions, as opposed to
extrapolating outside the trained positions, which may lead to catastrophic values. We verify our
approach theoretically, by showing that the interpolated attention score has a much smaller upper
bound (∼ 600× smaller in LLaMA 7B setting) than the extrapolated one, and is thus much more
stable. Therefore, interpolated position encodings are easier for the model to adapt.
Empirically, we found that Position Interpolation is highly effective and efficient, requiring only a
very short period of fine-tuning for the model to fully adapt to greatly extended context windows.
We present experimental results for extending the context window to up to 32768 from the initial
2048 across 7B to 65B LLaMA models using Position Interpolation. Our results show that

1. Position Interpolation can easily enable very long context windows (e.g. 32768), requiring
only fine-tuning for 1000 steps on the Pile (Gao et al., 2020) to achieve a good quality.
The cost of fine-tuning is negligible compared to the pre-training costs. This confirms
our hypothesis that it is relatively easy for the models to adapt to interpolated position
encodings.
2. Position Interpolation generates strong models that can effectively make use of much extended context window. We show that models extended by Position Interpolation enjoy
significant perplexity gains from greatly extended context windows for text modeling, and
we show that the perplexity reduces graceful with the enlargement of context windows.
We also applied Position Interpolation in a long text summarization task, and demonstrate
competitive performances.
3. Position Interpolation preserves model quality relatively well for tasks within its original
context window sizes. We present a variety of evaluation results for the extended LLaMA
models on the original LLaMA benchmark. Compared with original LLaMA models, the
extended LLaMA models saw a minor degradation on several standard benchmarks within
a 2048 token limit.
Our results highlight the innate ability of Transformer models to “extrapolate to sequence lengths longer than the ones encountered during training” as hypothesized in the seminal work of Vaswani et al. (2017). We reaffirm this hypothesis and suggest that the previously known weakness of extrapolating to longer sequences for language modeling (Press et al., 2022) may be due to direct extrapolation of positional encodings and it can be largely mitigated by interpolating position encodings instead.
Concurrent work. Right before our release, we are informed with a concurrent blogpost (SuperHOT kaiokendev (2023)) that also interpolates positional encoding in RoPE to extend the context window from 2K to 8K. Recently, open source community picks it up in Reddit post 1
and Github Issues 2, which shows that fine-tuning with LoRA (Hu et al., 2021) also seems to work well. Our paper shows a full fine-tuning with up to 65B model work well with Position Interpolation, and we also give theoretical explanations why interpolation achieves much more stable results than extrapolation, by showing that the upper bound of interplated attention score is much lower than that of extrapolated ones
---
Here is the summary of the article:
"""

middle_prompt2 = """
Text
---
LLaMA-2-7B-32K
Model Description

LLaMA-2-7B-32K is an open-source, long context language model developed by Together, fine-tuned from Meta's original Llama-2 7B model. This model represents our efforts to contribute to the rapid progress of the open-source ecosystem for large language models. The model has been extended to a context length of 32K with position interpolation, allowing applications on multi-document QA, long text summarization, etc.
What's new?

This model introduces several improvements and new features:

    Extended Context: The model has been trained to handle context lengths up to 32K, which is a significant improvement over the previous versions.

    Pre-training and Instruction Tuning: We have shared our data recipe, which consists of a mixture of pre-training and instruction tuning data.

    Fine-tuning Examples: We provide examples of how to fine-tune the model for specific applications, including book summarization and long context question and answering.

    Software Support: We have updated both the inference and training stack to allow efficient inference and fine-tuning for 32K context.

Model Architecture

The model follows the architecture of Llama-2-7B and extends it to handle a longer context. It leverages the recently released FlashAttention-2 and a range of other optimizations to improve the speed and efficiency of inference and training.
Training and Fine-tuning

The model has been trained using a mixture of pre-training and instruction tuning data.

    In the first training phase of continued pre-training, our data mixture contains 25% RedPajama Book, 25% RedPajama ArXiv (including abstracts), 25% other data from RedPajama, and 25% from the UL2 Oscar Data, which is a part of OIG (Open-Instruction-Generalist), asking the model to fill in missing chunks, or complete the text. To enhance the long-context ability, we exclude data shorter than 2K word. The inclusion of UL2 Oscar Data is effective in compelling the model to read and utilize long-range context.
    We then fine-tune the model to focus on its few shot capacity under long context, including 20% Natural Instructions (NI), 20% Public Pool of Prompts (P3), 20% the Pile. We decontaminated all data against HELM core scenarios . We teach the model to leverage the in-context examples by packing examples into one 32K-token sequence. To maintain the knowledge learned from the first piece of data, we incorporate 20% RedPajama-Data Book and 20% RedPajama-Data ArXiv.

Next, we provide examples of how to fine-tune the model for specific applications. The example datasets are placed in togethercomputer/Long-Data-Collections You can use the OpenChatKit to fine-tune your own 32K model over LLaMA-2-7B-32K. Please refer to OpenChatKit for step-by-step illustrations.

    Long Context QA.

    We take as an example the multi-document question answering task from the paper “Lost in the Middle: How Language Models Use Long Contexts”. The input for the model consists of (i) a question that requires an answer and (ii) k documents, which are passages extracted from Wikipedia. Notably, only one of these documents contains the answer to the question, while the remaining k − 1 documents, termed as "distractor" documents, do not. To successfully perform this task, the model must identify and utilize the document containing the answer from its input context.

    With OCK, simply run the following command to fine-tune:

    bash training/finetune_llama-2-7b-32k-mqa.sh

Summarization.

Another example is BookSum, a unique dataset designed to address the challenges of long-form narrative summarization. This dataset features source documents from the literature domain, including novels, plays, and stories, and offers human-written, highly abstractive summaries. We here focus on chapter-level data. BookSum poses a unique set of challenges, necessitating that the model comprehensively read through each chapter.

With OCK, simply run the following command to fine-tune:

bash training/finetune_llama-2-7b-32k-booksum.sh

Inference

You can use the Together API to try out LLaMA-2-7B-32K for inference. The updated inference stack allows for efficient inference.

To run the model locally, we strongly recommend to install Flash Attention V2, which is necessary to obtain the best performance:

# Please update the path of `CUDA_HOME`
export CUDA_HOME=/usr/local/cuda-11.8
pip install transformers==4.31.0
pip install sentencepiece
pip install ninja
pip install flash-attn --no-build-isolation
pip install git+https://github.com/HazyResearch/flash-attention.git#subdirectory=csrc/rotary

You can use this model directly from the Hugging Face Model Hub or fine-tune it on your own data using the OpenChatKit.

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("togethercomputer/LLaMA-2-7B-32K")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/LLaMA-2-7B-32K", trust_remote_code=True, torch_dtype=torch.float16)

input_context = "Your text here"
input_ids = tokenizer.encode(input_context, return_tensors="pt")
output = model.generate(input_ids, max_length=128, temperature=0.7)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

Alternatively, you can set trust_remote_code=False if you prefer not to use flash attention.
Limitations and Bias

As with all language models, LLaMA-2-7B-32K may generate incorrect or biased content. It's important to keep this in mind when using the model.
Community

Join us on Together Discord
---
Here is the summary of the text above:

"""

long_prompt = """
Text
---
Transformers

State-of-the-art Machine Learning for PyTorch, TensorFlow, and JAX.

Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. These models support common tasks in different modalities, such as:

Natural Language Processing: text classification, named entity recognition, question answering, language modeling, summarization, translation, multiple choice, and text generation.
Computer Vision: image classification, object detection, and segmentation.
Audio: automatic speech recognition and audio classification.
Multimodal: table question answering, optical character recognition, information extraction from scanned documents, video classification, and visual question answering.

🤗 Transformers support framework interoperability between PyTorch, TensorFlow, and JAX. This provides the flexibility to use a different framework at each stage of a model’s life; train a model in three lines of code in one framework, and load it for inference in another. Models can also be exported to a format like ONNX and TorchScript for deployment in production environments.

Join the growing community on the Hub, forum, or Discord today!
If you are looking for custom support from the Hugging Face team
HuggingFace Expert Acceleration Program
Contents

The documentation is organized into five sections:

    GET STARTED provides a quick tour of the library and installation instructions to get up and running.

    TUTORIALS are a great place to start if you’re a beginner. This section will help you gain the basic skills you need to start using the library.

    HOW-TO GUIDES show you how to achieve a specific goal, like finetuning a pretrained model for language modeling or how to write and share a custom model.

    CONCEPTUAL GUIDES offers more discussion and explanation of the underlying concepts and ideas behind models, tasks, and the design philosophy of 🤗 Transformers.

    API describes all classes and functions:
        MAIN CLASSES details the most important classes like configuration, model, tokenizer, and pipeline.
        MODELS details the classes and functions related to each model implemented in the library.
        INTERNAL HELPERS details utility classes and functions used internally.

Supported models

    ALBERT (from Google Research and the Toyota Technological Institute at Chicago) released with the paper ALBERT: A Lite BERT for Self-supervised Learning of Language Representations, by Zhenzhong Lan, Mingda Chen, Sebastian Goodman, Kevin Gimpel, Piyush Sharma, Radu Soricut.
    ALIGN (from Google Research) released with the paper Scaling Up Visual and Vision-Language Representation Learning With Noisy Text Supervision by Chao Jia, Yinfei Yang, Ye Xia, Yi-Ting Chen, Zarana Parekh, Hieu Pham, Quoc V. Le, Yunhsuan Sung, Zhen Li, Tom Duerig.
    AltCLIP (from BAAI) released with the paper AltCLIP: Altering the Language Encoder in CLIP for Extended Language Capabilities by Chen, Zhongzhi and Liu, Guang and Zhang, Bo-Wen and Ye, Fulong and Yang, Qinghong and Wu, Ledell.
    Audio Spectrogram Transformer (from MIT) released with the paper AST: Audio Spectrogram Transformer by Yuan Gong, Yu-An Chung, James Glass.
    Autoformer (from Tsinghua University) released with the paper Autoformer: Decomposition Transformers with Auto-Correlation for Long-Term Series Forecasting by Haixu Wu, Jiehui Xu, Jianmin Wang, Mingsheng Long.
    Bark (from Suno) released in the repository suno-ai/bark by Suno AI team.
    BART (from Facebook) released with the paper BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension by Mike Lewis, Yinhan Liu, Naman Goyal, Marjan Ghazvininejad, Abdelrahman Mohamed, Omer Levy, Ves Stoyanov and Luke Zettlemoyer.
    BARThez (from École polytechnique) released with the paper BARThez: a Skilled Pretrained French Sequence-to-Sequence Model by Moussa Kamal Eddine, Antoine J.-P. Tixier, Michalis Vazirgiannis.
    BARTpho (from VinAI Research) released with the paper BARTpho: Pre-trained Sequence-to-Sequence Models for Vietnamese by Nguyen Luong Tran, Duong Minh Le and Dat Quoc Nguyen.
    BEiT (from Microsoft) released with the paper BEiT: BERT Pre-Training of Image Transformers by Hangbo Bao, Li Dong, Furu Wei.
    BERT (from Google) released with the paper BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding by Jacob Devlin, Ming-Wei Chang, Kenton Lee and Kristina Toutanova.
    BERT For Sequence Generation (from Google) released with the paper Leveraging Pre-trained Checkpoints for Sequence Generation Tasks by Sascha Rothe, Shashi Narayan, Aliaksei Severyn.
    BERTweet (from VinAI Research) released with the paper BERTweet: A pre-trained language model for English Tweets by Dat Quoc Nguyen, Thanh Vu and Anh Tuan Nguyen.
    BigBird-Pegasus (from Google Research) released with the paper Big Bird: Transformers for Longer Sequences by Manzil Zaheer, Guru Guruganesh, Avinava Dubey, Joshua Ainslie, Chris Alberti, Santiago Ontanon, Philip Pham, Anirudh Ravula, Qifan Wang, Li Yang, Amr Ahmed.
    BigBird-RoBERTa (from Google Research) released with the paper Big Bird: Transformers for Longer Sequences by Manzil Zaheer, Guru Guruganesh, Avinava Dubey, Joshua Ainslie, Chris Alberti, Santiago Ontanon, Philip Pham, Anirudh Ravula, Qifan Wang, Li Yang, Amr Ahmed.
    BioGpt (from Microsoft Research AI4Science) released with the paper BioGPT: generative pre-trained transformer for biomedical text generation and mining by Renqian Luo, Liai Sun, Yingce Xia, Tao Qin, Sheng Zhang, Hoifung Poon and Tie-Yan Liu.
    BiT (from Google AI) released with the paper Big Transfer (BiT): General Visual Representation Learning by Alexander Kolesnikov, Lucas Beyer, Xiaohua Zhai, Joan Puigcerver, Jessica Yung, Sylvain Gelly, Neil Houlsby.
    Blenderbot (from Facebook) released with the paper Recipes for building an open-domain chatbot by Stephen Roller, Emily Dinan, Naman Goyal, Da Ju, Mary Williamson, Yinhan Liu, Jing Xu, Myle Ott, Kurt Shuster, Eric M. Smith, Y-Lan Boureau, Jason Weston.
    BlenderbotSmall (from Facebook) released with the paper Recipes for building an open-domain chatbot by Stephen Roller, Emily Dinan, Naman Goyal, Da Ju, Mary Williamson, Yinhan Liu, Jing Xu, Myle Ott, Kurt Shuster, Eric M. Smith, Y-Lan Boureau, Jason Weston.
    BLIP (from Salesforce) released with the paper BLIP: Bootstrapping Language-Image Pre-training for Unified Vision-Language Understanding and Generation by Junnan Li, Dongxu Li, Caiming Xiong, Steven Hoi.
    BLIP-2 (from Salesforce) released with the paper BLIP-2: Bootstrapping Language-Image Pre-training with Frozen Image Encoders and Large Language Models by Junnan Li, Dongxu Li, Silvio Savarese, Steven Hoi.
    BLOOM (from BigScience workshop) released by the BigScience Workshop.
    BORT (from Alexa) released with the paper Optimal Subarchitecture Extraction For BERT by Adrian de Wynter and Daniel J. Perry.
    BridgeTower (from Harbin Institute of Technology/Microsoft Research Asia/Intel Labs) released with the paper BridgeTower: Building Bridges Between Encoders in Vision-Language Representation Learning by Xiao Xu, Chenfei Wu, Shachar Rosenman, Vasudev Lal, Wanxiang Che, Nan Duan.
    ByT5 (from Google Research) released with the paper ByT5: Towards a token-free future with pre-trained byte-to-byte models by Linting Xue, Aditya Barua, Noah Constant, Rami Al-Rfou, Sharan Narang, Mihir Kale, Adam Roberts, Colin Raffel.
    CamemBERT (from Inria/Facebook/Sorbonne) released with the paper CamemBERT: a Tasty French Language Model by Louis Martin, Benjamin Muller, Pedro Javier Ortiz Suárez*, Yoann Dupont, Laurent Romary, Éric Villemonte de la Clergerie, Djamé Seddah and Benoît Sagot.
    CANINE (from Google Research) released with the paper CANINE: Pre-training an Efficient Tokenization-Free Encoder for Language Representation by Jonathan H. Clark, Dan Garrette, Iulia Turc, John Wieting.
    Chinese-CLIP (from OFA-Sys) released with the paper Chinese CLIP: Contrastive Vision-Language Pretraining in Chinese by An Yang, Junshu Pan, Junyang Lin, Rui Men, Yichang Zhang, Jingren Zhou, Chang Zhou.
    CLAP (from LAION-AI) released with the paper Large-scale Contrastive Language-Audio Pretraining with Feature Fusion and Keyword-to-Caption Augmentation by Yusong Wu, Ke Chen, Tianyu Zhang, Yuchen Hui, Taylor Berg-Kirkpatrick, Shlomo Dubnov.
    CLIP (from OpenAI) released with the paper Learning Transferable Visual Models From Natural Language Supervision by Alec Radford, Jong Wook Kim, Chris Hallacy, Aditya Ramesh, Gabriel Goh, Sandhini Agarwal, Girish Sastry, Amanda Askell, Pamela Mishkin, Jack Clark, Gretchen Krueger, Ilya Sutskever.
    CLIPSeg (from University of Göttingen) released with the paper Image Segmentation Using Text and Image Prompts by Timo Lüddecke and Alexander Ecker.
    CodeGen (from Salesforce) released with the paper A Conversational Paradigm for Program Synthesis by Erik Nijkamp, Bo Pang, Hiroaki Hayashi, Lifu Tu, Huan Wang, Yingbo Zhou, Silvio Savarese, Caiming Xiong.

Supported frameworks

The table below represents the current support in the library for each of those models, whether they have a Python tokenizer (called “slow”). A “fast” tokenizer backed by the 🤗 Tokenizers library, whether they have support in Jax (via Flax), PyTorch, and/or TensorFlow.
---
Here is the main content and summary of the article above, including the main points, the last supported model and the list of supported frameworks:

"""

In [2]:
# load model
import os
os.environ["HF_HOME"] = "/data/cache/huggingface"
os.environ["CUDA_HOME"] = "/usr/local/cuda-11.7"
# check env
print('HF_HOME: {}'.format(os.environ["HF_HOME"]))

import torch
print(torch.cuda.is_available())


HF_HOME: /data/cache/huggingface
True


The above cells are always executed

In [3]:
from transformers import LlamaForCausalLM, LlamaTokenizer
MODEL = "huggyllama/llama-7b"
llama_7b_tokenizer = LlamaTokenizer.from_pretrained(MODEL)

[2023-08-14 20:56:25,362] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [4]:
# count
short_ids = llama_7b_tokenizer.encode(short_prompt, return_tensors="pt").to('cuda')
middle_ids = llama_7b_tokenizer.encode(middle_prompt, return_tensors="pt").to('cuda')
long_ids = llama_7b_tokenizer.encode(long_prompt, return_tensors="pt").to('cuda')

print(short_ids.shape[1])
print(middle_ids.shape[1])
print(long_ids.shape[1])

10
1318
835


In [3]:

llama7b_model = LlamaForCausalLM.from_pretrained(MODEL, device_map="auto")

# unwind broken decapoda-research config
llama7b_model.config.pad_token_id = llama_7b_tokenizer.pad_token_id = 0  # unk
llama7b_model.config.bos_token_id = 1
llama7b_model.config.eos_token_id = 2


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# try self-defined flash model

from modeling_llama import FlashLlamaForCausalLM
from transformers import LlamaTokenizer, LlamaConfig
flash_model = FlashLlamaForCausalLM.from_pretrained("huggyllama/llama-7b", device_map="auto")
flash_tokenizer = LlamaTokenizer.from_pretrained("huggyllama/llama-7b")

# unwind broken decapoda-research config
flash_model.config.pad_token_id = flash_tokenizer.pad_token_id = 0  # unk
flash_model.config.bos_token_id = 1
flash_model.config.eos_token_id = 2

>>>> Flash Attention installed


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(llama7b_model.config)
print(flash_model.config)

LlamaConfig {
  "_name_or_path": "huggyllama/llama-7b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "max_sequence_length": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}

LlamaConfig {
  "_name_or_path": "huggyllama/llama-7b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "max_sequence_length": 2048,
  "model_type": "

In [3]:
# 32K
# Load model directly
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

llama_7b_32k_tokenizer = AutoTokenizer.from_pretrained("togethercomputer/LLaMA-2-7B-32K")
llama7b_32k_model = AutoModelForCausalLM.from_pretrained("togethercomputer/LLaMA-2-7B-32K", device_map="auto", trust_remote_code=True, torch_dtype=torch.float16)

>>>> Flash Attention installed
>>>> Flash RoPE installed


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# or run this for traditional attention
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

llama_7b_32k_tokenizer = AutoTokenizer.from_pretrained("togethercomputer/LLaMA-2-7B-32K")
llama7b_32k_model = AutoModelForCausalLM.from_pretrained("togethercomputer/LLaMA-2-7B-32K", device_map="auto", torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

prompt = "Here is an introduction of the city LA:"

# encode context the generation is conditioned on
input_ids = llama_7b_tokenizer.encode(prompt, return_tensors="pt").to('cuda')

print(input_ids)

input_ids1 = llama_7b_32k_tokenizer.encode(prompt, return_tensors="pt").to('cuda')

print(input_ids1)       # no bos for 32k?

tensor([[    1,  2266,   338,   385, 18707,   310,   278,  4272, 17900, 29901]],
       device='cuda:0')
tensor([[ 2266,   338,   385, 18707,   310,   278,  4272, 17900, 29901]],
       device='cuda:0')


### Try different text length

In [6]:
# togethercomputer implementation
import time
start = time.time()
llama7b_32k_gen_ids = llama7b_32k_model.generate(short_ids, max_length=800)
end = time.time()
print(llama_7b_32k_tokenizer.decode(llama7b_32k_gen_ids[0][len(short_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
print(end - start)

start = time.time()
llama7b_32k_gen_ids = llama7b_32k_model.generate(middle_ids, max_length=2000)
end = time.time()
print(llama_7b_32k_tokenizer.decode(llama7b_32k_gen_ids[0][len(middle_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
print(end - start)
























































































































































































































































































































































































































































































































































































































































































































































































































33.83553647994995

1. Position Interpolation can easily enable very long context windows (e.g. 32768), requiring
only fine-tuning for 1000 steps on the Pile (Gao et al., 2020) to achieve a good quality.
The co

In [ ]:
# generate and decode
import time

# short
start = time.time() 
llama7b_gen_ids = llama7b_model.generate(short_ids, max_length=800)
end = time.time()
# only decode the generated ids, strip the prompt
print(llama_7b_tokenizer.decode(llama7b_gen_ids[0][len(short_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
print(end - start)

# start = time.time()
# llama7b_32k_gen_ids = llama7b_32k_model.generate(short_ids, max_length=800)
# end = time.time()
# print(llama_7b_32k_tokenizer.decode(llama7b_32k_gen_ids[0][len(short_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
# print(end - start)

start = time.time()
flash_gen_ids = flash_model.generate(short_ids, max_length=800)
end = time.time()
print(flash_tokenizer.decode(flash_gen_ids[0][len(short_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
print(end - start)



The city of Los Angeles is the second largest city in the United States, after New York City. It is the center of the Los Angeles metropolitan area, which is the second largest metropolitan area in the United States, after the New York metropolitan area.
Los Angeles is located in Southern California, in the southwest corner of the United States. It is bordered by the Pacific Ocean on the southwest, by the San Gabriel Mountains on the northeast, by the San Fernando Valley on the east, and by the Santa Monica Mountains on the southeast.
Los Angeles is the largest city in the United States that is not on a river, lake, or the sea.
Los Angeles is the largest city in the United States that is not on a river, lake, or the sea. It is also the largest city in the United States that is not on the coast.
Los Angeles is the largest city in the United States that is not on a river, lake, or the sea. It is also the largest city in the United States that is not on the coast. Los Angeles is the seco

/opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [213,0,0], thread: [32,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [213,0,0], thread: [33,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [213,0,0], thread: [34,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [213,0,0], thread: [35,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/cuda/Sc

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# middle
start = time.time()
llama7b_gen_ids = llama7b_model.generate(middle_ids, max_length=2000)
end = time.time()
print(llama_7b_tokenizer.decode(llama7b_gen_ids[0][len(middle_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
print(end - start)

# start = time.time()
# llama7b_32k_gen_ids = llama7b_32k_model.generate(middle_ids, max_length=2000)
# end = time.time()
# print(llama_7b_32k_tokenizer.decode(llama7b_32k_gen_ids[0][len(middle_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
# print(end - start)

start = time.time()
flash_gen_ids = flash_model.generate(middle_ids, max_length=2000)
end = time.time()
print(flash_tokenizer.decode(flash_gen_ids[0][len(middle_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
print(end - start)


We introduce Position Interpolation, a technique to extend the context window of pre-trained Transformer models.
Position Interpolation interpolates the position encodings at neighboring integer positions, utilizing the fact that position encodings can be applied on non-integer positions, as opposed to extrapolating outside the trained positions, which may lead to catastrophic values.
We verify our approach theoretically, by showing that the interpolated attention score has a much smaller upper bound than the extrapolated one, and is thus much more stable.
Empirically, we found that Position Interpolation is highly effective and efficient, requiring only a very short period of fine-tuning for the model to fully adapt to greatly extended context windows.
We present experimental results for extending the context window to up to 32768 from the initial 2048 across 7B to 65B LLaMA models using Position Interpolation.
Our results show that Position Interpolation can easily enable very long 

In [ ]:
# long, not available for base
# start = time.time()
# llama7b_32k_gen_ids = llama7b_32k_model.generate(long_ids, max_length=4000)
# end = time.time()
# print(llama_7b_32k_tokenizer.decode(llama7b_32k_gen_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False))
# print(end - start)

Try 65b model with flash 

In [2]:
# 65b no flash
from transformers import LlamaForCausalLM
model_65b = LlamaForCausalLM.from_pretrained("huggyllama/llama-65b", device_map="auto")

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [13]:
# 65b no flash
from modeling_llama_raw import LlamaForCausalLM
model_65b = LlamaForCausalLM.from_pretrained("huggyllama/llama-65b", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# not executed together with the above cell
from modeling_llama import FlashLlamaForCausalLM
from transformers import LlamaTokenizer, LlamaConfig
model_65b = FlashLlamaForCausalLM.from_pretrained("huggyllama/llama-65b", device_map="auto")

>>>> Flash Attention installed


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [1]:
# from together computer
import torch
from modeling_flash_llama_raw import LlamaForCausalLM as LlamaForCausalLM1

model_65b = LlamaForCausalLM1.from_pretrained("huggyllama/llama-65b", device_map="auto", torch_dtype=torch.float16)

>>>> Flash Attention installed
>>>> Flash RoPE installed


KeyboardInterrupt: 

In [2]:
# from together computer
import torch
from modeling_flash_llama import LlamaForCausalLM as LlamaForCausalLM2

model_65b = LlamaForCausalLM2.from_pretrained("huggyllama/llama-65b", device_map="auto", torch_dtype=torch.float16)

>>>> Flash Attention installed


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
from transformers import LlamaTokenizer
tokenizer_65b = LlamaTokenizer.from_pretrained("huggyllama/llama-65b")

# unwind broken decapoda-research config
model_65b.config.pad_token_id = tokenizer_65b.pad_token_id = 0  # unk
model_65b.config.bos_token_id = 1
model_65b.config.eos_token_id = 2


In [6]:

short_prompt = "Here is an introduction of the city LA:"
from transformers import LlamaTokenizer
tokenizer_65b = LlamaTokenizer.from_pretrained("huggyllama/llama-65b")
# count
short_ids = tokenizer_65b.encode(short_prompt, return_tensors="pt").to('cuda')
middle_ids = tokenizer_65b.encode(middle_prompt, return_tensors="pt").to('cuda')
long_ids = tokenizer_65b.encode(long_prompt, return_tensors="pt").to('cuda')

print(short_ids.shape[1])
print(middle_ids.shape[1])
print(long_ids.shape[1])

10
1318
830


In [7]:
# short
import time
start = time.time() 
llama65b_gen_ids = model_65b.generate(short_ids, max_length=800)
end = time.time()
# only decode the generated ids, strip the prompt
print(tokenizer_65b.decode(llama65b_gen_ids[0][len(short_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
print(end - start)
print("Generated length: ", len(llama65b_gen_ids[0]))


Los Angeles is the second largest city in the United States, with a population of nearly four million people. It is located in southern California, and is the center of the nation's film and television industry.
Los Angeles is a sprawling metropolis, and visitors should be prepared to drive long distances if they wish to see the sights. The city is also famous for its smog, which is at its worst during the summer months.
The city is divided into a number of districts, including Downtown, Hollywood, the Westside, the San Fernando Valley and the Harbor Area.
Downtown Los Angeles is the center of the city's government and business, and is home to the Los Angeles Convention Center, the Civic Center, and the Staples Center sports arena.
Hollywood is the center of the entertainment industry in Los Angeles, and is home to the Hollywood Walk of Fame, Grauman's Chinese Theater, and the Hollywood Bowl.
The Westside is home to the city's most exclusive neighborhoods, and is also the center of th

In [9]:
# middle
start = time.time() 
llama65b_gen_ids = model_65b.generate(middle_ids, max_length=2000)
end = time.time()
# only decode the generated ids, strip the prompt
print(tokenizer_65b.decode(llama65b_gen_ids[0][len(middle_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
print(end - start)
print("Generated length: ", len(llama65b_gen_ids[0]))


1. Position Interpolation can easily enable very long context windows (e.g. 32768), requiring
only fine-tuning for 1000 steps on the Pile (Gao et al., 2020) to achieve a good quality.
The cost of fine-tuning is negligible compared to the pre-training costs. This confirms
our hypothesis that it is relatively easy for the models to adapt to interpolated position
encodings.
2. Position Interpolation generates strong models that can effectively make use of much extended context window. We show that models extended by Position Interpolation enjoy
significant perplexity gains from greatly extended context windows for text modeling, and
we show that the perplexity reduces graceful with the enlargement of context windows.
We also applied Position Interpolation in a long text summarization task, and demonstrate
competitive performances.
3. Position Interpolation preserves model quality relatively well for tasks within its original
context window sizes. We present a variety of evaluation result

In [11]:
# middle
start = time.time() 
llama65b_gen_ids = model_65b.generate(long_ids, max_length=10000)
end = time.time()
# only decode the generated ids, strip the prompt
print(tokenizer_65b.decode(llama65b_gen_ids[0][len(long_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))
print(end - start)
print("Generated length: ", len(llama65b_gen_ids[0]))

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


KeyboardInterrupt: 

### Try RoPE Scaling

In [ ]:
# 7b, linear
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
linear4_model = LlamaForCausalLM.from_pretrained("huggyllama/llama-7b", rope_scaling={
    "type": "linear",
    "factor": 4,    # for 8k
})

linear4_tokenizer = LlamaTokenizer.from_pretrained("huggyllama/llama-7b")

print(linear4_model.config)

In [ ]:
# ids
short_ids = linear4_tokenizer.encode(short_prompt, return_tensors="pt").to("cuda")
middle_ids = linear4_tokenizer.encode(middle_prompt, return_tensors="pt").to("cuda")
long_ids = linear4_tokenizer.encode(long_prompt, return_tensors="pt").to("cuda")

# short 
import time
start = time.time()
short_output = linear4_model.generate(short_ids, max_length=800)
end = time.time()
print(end - start)
print(linear4_tokenizer.decode(short_output[0][len(short_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))

# middle
start = time.time()
middle_output = linear4_model.generate(middle_ids, max_length=2000)
end = time.time()
print(end - start)
print(linear4_tokenizer.decode(middle_output[0][len(middle_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))

# long
start = time.time()
long_output = linear4_model.generate(long_ids, max_length=4000)
end = time.time()
print(end - start)
print(linear4_tokenizer.decode(long_output[0][len(long_ids[0]):], skip_special_tokens=True, clean_up_tokenization_spaces=False))

In [3]:
%pip install pytest

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [6]:
import math

import torch
import torch.nn.functional as F
import pytest

from einops import rearrange

from flash_attn.layers.rotary import apply_rotary_emb_func, apply_rotary_emb_torch


is_sm8x = torch.cuda.get_device_capability('cuda') >= (8, 0)

@pytest.mark.parametrize('dtype', ([torch.float16] if not is_sm8x else [torch.float16, torch.bfloat16]))
# @pytest.mark.parametrize('dtype', ([torch.float16]))
@pytest.mark.parametrize('rotary_fraction', [1.0, 0.5])
# @pytest.mark.parametrize('rotary_fraction', [0.5])
@pytest.mark.parametrize('inplace', [False, True])
# @pytest.mark.parametrize('inplace', [False])
def test_rotary_single_tensor(inplace, rotary_fraction, dtype):
    rtol = 1e-3
    batch_size = 32
    nheads = 4
    seqlen = 217
    headdim = 128
    x = torch.randn(batch_size, seqlen, nheads, headdim, dtype=dtype, device='cuda',
                    requires_grad=True)
    x_pt = x.detach().clone().requires_grad_()
    rotary_dim = int(rotary_fraction * headdim)
    assert rotary_dim % 2 == 0
    angle = torch.randn(seqlen, rotary_dim // 2, device='cuda')
    cos = torch.cos(angle).to(dtype=dtype)
    sin = torch.sin(angle).to(dtype=dtype)
    out = apply_rotary_emb_func(x, cos, sin, inplace)
    print(out.shape)
    out_pt = apply_rotary_emb_torch(x_pt, cos, sin)
    # Numerical error if we just do any arithmetic
    atol = ((out + 0.3 - 0.3) - out).abs().max().item()
    assert torch.allclose(out, out_pt, rtol=rtol, atol=2 * atol)
    g = torch.randn_like(out)
    g_pt = g.clone()  # If inplace=True, we might modify the gradient inplace
    out.backward(g)
    out_pt.backward(g_pt)
    atol = ((x_pt.grad + 0.3 - 0.3) - x_pt.grad).abs().max().item()
    assert torch.allclose(x.grad, x_pt.grad, rtol=rtol, atol=2 * atol)

test_rotary_single_tensor(inplace=True, rotary_fraction=1.0, dtype=torch.float16)

torch.Size([32, 217, 4, 128])


AssertionError: 